In [ ]:
using Pkg; Pkg.activate("..")

In [ ]:
import Distributions: Normal, MvNormal, Exponential, logpdf, pdf
import Plots
import PyPlot
import Statistics: mean, std

In [ ]:

"""
Slice sampling

    References
    Iain Murray's matlab impl for multivariate slice sampling:
        https://homepages.inf.ed.ac.uk/imurray2/teaching/09mlss/slice_sample.m
    David MacKay's book:
        http://www.inference.org.uk/itprnn/book.pdf
    Radford Neal's R impl for univariate slice sampling:
        https://www.cs.toronto.edu/~radford/ftp/slice-R-prog
    Alternative impl that is not well supported, thorough:
        https://github.com/brian-j-smith/Mamba.jl/blob/release-0.11/src/samplers/slice.jl

x0: Initial point
f: The density function to sample from
N: The number of simulate samples

step_limit (m): Limit on steps (default Inf)
width (w): The size of step for creating interval [L, R] (defaults to 1)
burn: The number of samples to discard before saving (defaults to 0)
thin: Discards `thin-1` samples and return the next to avoid correlation (defaults to 1)
logpdf: `true` if g is the log of probability density trying to sample from (defaults to `false`)

"""
function slicesample(
    x0::Array{Float64, 1},
    f,
    N::Int;
    step_limit::Int = typemax(Int),
    width::Union{Array{Float64, 1}, Float64} = 1.,
    burn::Int = 0,
    thin::Int = 1,
    logpdf::Bool = false)

    D = size(x0, 1)
    if isa(width, Float64)
        width = fill(width, D)
    end

    if logpdf == false
        g(x) = log(f(x))
    else
        g = f
    end


    L = zeros(D)
    R = zeros(D)
    x = zeros(D)
    x′= zeros(D)
    samples = zeros(D, N)

    x[:] = x0

    # Log density at initial point
    gx = g(x)

    for i in 1:(N*thin+burn)

        # Determine the slice level, in log terms.
        logy = log(rand()) + gx

        # Sample from each dimension separately
        for d in 1:D
            x′[:] = x
            L[:] = x
            R[:] = x

            # Find interval x ∈ [L, R] to sample from
            u = width[d] * rand()
            L[d] = x[d] - u
            R[d] = x[d] + (width[d] - u)

            # Step out
            if step_limit == Inf
                while (g(L) > logy)
                    L[d] -= width[d]
                end

                while (g(R) > logy)
                    R[d] += width[d]
                end
            elseif step_limit > 1
                J = floor(step_limit * rand())
                K = (step_limit - 1) - J

                while (J > 0) && (g(L) > logy)
                    L[d] -= width[d]
                    J -= 1
                end

                while (K > 0) && (g(R) > logy)
                    R[d] += width[d]
                    K -= 1
                end
            end

            # Sample from [L, R], shrinking it on each rejection
            #     until we have found good sample x' in the slice
            while true
                x′[d] = L[d] + rand() * (R[d] - L[d])
                gx = g(x′)

                gx >= logy && break

                if x′[d] > x[d]
                    R[d] = x′[d]
                else
                    L[d] = x′[d]
                end
            end

            x[d] = x′[d]
        end

        if i > burn && ((i-burn) % thin == 0)
            samples[:, div(i-burn, thin)] = x[:]
        end
    end

    samples
end


In [ ]:
x0 = [0.]
g(x) = pdf(Normal(), x[1])
N = 5000

samples = slicesample(x0, g, N; step_limit=10, width=0.5, burn=20, thin=1)
samples

In [ ]:
print("mean: $(mean(samples))  std: $(std(samples))")

In [ ]:
PyPlot.plt[:hist](vec(samples), bins=20, density=true)

xs = -3:0.01:3
ys = [pdf(Normal(), x) for x in xs]

PyPlot.plt[:plot](xs, ys, color="red", linestyle="--")

In [ ]:
x0 = zeros(2)
g(x) = pdf(MvNormal(2, 1), x)
n = 5000

samples = slicesample(x0, g, n; step_limit=10, width=1., burn=20, thin=1)
samples

In [ ]:
print("mean: $(mean(samples))  std: $(std(samples))")

In [ ]:
PyPlot.plt[:hist2d](samples[1,:], samples[2,:], bins=20)
""

In [ ]:
using CreditRisk

In [ ]:
import LinearAlgebra: mul!

" Computes μ for the approximating standard normal of loss probability"
function approx_μ(pnc, weights, A)
    @. A = pnc * weights
    sum(A)
end


" Computes σ for the approximating standard normal of loss probability"
function approx_σ(pnc, lgc, ead, A, B, C)
    n, c = size(lgc)
    i = 1

    lgcs = Vector{SubArray}(undef, c)
    pncs = Vector{SubArray}(undef, c)
    for idx = 1:c
        lgcs[idx] = @view lgc[:,idx]
        pncs[idx] = @view pnc[:,idx]
    end

    for a = 1:c
        for b = 1:(a-1)
            @. A[:,i] = (lgcs[a] - lgcs[b])^2 * pncs[a] * pncs[b]
            i += 1
        end
    end

    sum!(B, A)
    @. C = ead^2 * B
    sqrt(sum(C) / sum(ead)^2)
end


In [ ]:
n = 2500
c = 4
s = 1
l = 0.2
param = Parameter(n,c,s,l)
(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

In [ ]:
Z = zeros(S)
βZ = zeros(N)
phi0 = zeros(N, C+1)
phi  = @view phi0[:,2:end]
pnc = zeros(N, C)
approx_μ_A = similar(weights)
approx_σ_A = zeros(N, convert(Int, (C-1)*C/2))
approx_σ_B = zeros(N)
approx_σ_C = zeros(N)
Zdist = MvNormal(S, 1)

function π(Z)
    mul!(βZ, β, Z)
    @. phi = normcdf((H - βZ) / denom)
    diff!(pnc, phi0; dims=2)
    μ = approx_μ(pnc, weights, approx_μ_A)
    σ = approx_σ(pnc, lgc, ead, approx_σ_A, approx_σ_B, approx_σ_C)
    p = (1 - normcdf((l-μ)/σ)) * pdf(Zdist, Z)
    return p
end


@time samp = slicesample(zeros(S), π, 1000; step_limit=30, width=2., burn=100, thin=2)
PyPlot.plt[:hist](vec(samp), bins=20, density=true)

xs = -5:0.1:5
ys = [π([x]) for x in xs]

PyPlot.plt[:plot](xs, ys, color="red", linestyle="--")

""

In [ ]:
n = 2500
c = 4
s = 2
l = 0.2
param = Parameter(n,c,s,l)
(N, C, S, l, cmm, ead, lgc, cn, β, H, denom, weights) = unpack(param)

In [ ]:
Z = zeros(S)
βZ = zeros(N)
phi0 = zeros(N, C+1)
phi  = @view phi0[:,2:end]
pnc = zeros(N, C)
approx_μ_A = similar(weights)
approx_σ_A = zeros(N, convert(Int, (C-1)*C/2))
approx_σ_B = zeros(N)
approx_σ_C = zeros(N)
Zdist = MvNormal(S, 1)

function π(Z)
    mul!(βZ, β, Z)
    @. phi = normcdf((H - βZ) / denom)
    diff!(pnc, phi0; dims=2)
    μ = approx_μ(pnc, weights, approx_μ_A)
    σ = approx_σ(pnc, lgc, ead, approx_σ_A, approx_σ_B, approx_σ_C)
    p = (1 - normcdf((l-μ)/σ)) * pdf(Zdist, Z)
    return p
end

n_samples = 600
burn_ratio = 0.1
initial_point = zeros(S)

samples = slicesample(initial_point, π, n_samples;
    step_limit=20,
    width=0.5,
    burn=Int(burn_ratio * n_samples),
    thin=3)

PyPlot.plt[:hist2d](samp[1,:], samp[2,:], bins=20)
""